In [73]:
import pandas as pd
import pymysql
import numpy as np
import datetime
import os

con = pymysql.connect('localhost', 'root', 'Karelia', 'geo_data')

In [200]:
def merge_csv(files):
    files = os.listdir('.')
    file_list = []

    for file in files:
        if 'csv' in file:
            name = file[:9] 
            name = pd.read_csv(file)
            file_list.append(name)
            
    merged = pd.concat(file_list)
    merged = merged.reset_index(drop=True).sort_values(['Vuosi','Kk','Pv'])

    merged['timestamp'] = pd.to_datetime(merged[merged.columns[0]].astype(str) +'-'+ merged[merged.columns[1]].astype(str)  +'-'+ merged[merged.columns[2]].astype(str) +' '+ merged[merged.columns[3]].astype(str))
    merged = merged.drop(columns=list(merged.columns[:5]))
    merged = merged.drop(merged.columns[4], axis=1)

    merged = merged.rename(columns={merged.columns[0]:'clouds',merged.columns[1]:'msl',merged.columns[2]:'precip mm',merged.columns[3]:'humidity',merged.columns[4]:'snow',merged.columns[5]:'temp',merged.columns[6]:'dew point',merged.columns[7]:'visibility',merged.columns[8]:'wind dir',merged.columns[9]:'wind gust',merged.columns[10]:'wind speed'})

    merged.loc[merged['snow'].str.contains("^-$",na=False,regex=True),'snow'] = 0
    merged['snow'] = merged['snow'].fillna(0).astype(int)
    merged.loc[merged['snow'] < 0,'snow']= 0

    merged.loc[merged['clouds'] > 8,'clouds'] = 8

    merged.loc[merged['precip mm'].str.contains("^-$",na=False,regex=True),'precip mm']  = 0
    merged['precip mm'] = merged['precip mm'].astype(float)
    merged.loc[merged['precip mm'] < 0,'precip mm']= 0

    merged['clouds'] = merged['clouds'] * 12.5

    merged = merged.drop_duplicates()
    merged = merged[['timestamp','temp','dew point','humidity','precip mm','msl','clouds','visibility','wind speed','wind gust','wind dir','snow']]
    merged['precip mm'] = merged['precip mm'].fillna(0)
    merged = merged.fillna(method='pad')
    
    return merged

In [202]:
merged = merge_csv(os.listdir('.'))

In [203]:
def update_db(merged):
    con.connect()
    insert_main = con.cursor()
    
    for record,temp,dew,humid,precip, msl,clouds,visible,wind_speed,wind_gust,wind_dir,snow in merged.values:
        insert_main.callproc('vantaa_weather_insert',(str(record),temp,dew,humid,precip, msl,clouds,visible,wind_speed,wind_gust,wind_dir,snow))

    con.commit()

In [204]:
update_db(merged)